In [ ]:
import pandas as pd

data = pd.read_csv("1_CancerGov_QA.csv")

In [ ]:
from sklearn.model_selection import train_test_split
data_cleaned = data.drop(['Unnamed: 0.1', 'Unnamed: 0'], axis=1)
train_set, remaining_set = train_test_split(data_cleaned, test_size=0.2, random_state=42)
validation_set, test_set = train_test_split(remaining_set, test_size=0.5, random_state=42)
train_set.shape, validation_set.shape, test_set.shape

((583, 3), (73, 3), (73, 3))

In [ ]:
import numpy as np
import random
import re
import nltk
from nltk.tokenize import sent_tokenize

nltk.download('punkt')

train_set_shuffled = train_set.sample(frac=1, random_state=42).reset_index(drop=True)
validation_set_shuffled = validation_set.sample(frac=1, random_state=42).reset_index(drop=True)

combined_data = pd.concat([train_set_shuffled, validation_set_shuffled]).reset_index(drop=True)

def clean_text(text):
    cleaned_text = text.lstrip('\n')
    cleaned_text = text.replace('\t', '')
    cleaned_text = re.sub(' +', ' ', cleaned_text)
    cleaned_text = re.sub(' +', ' ', cleaned_text)
    cleaned_text = re.sub(' +', ' ', cleaned_text)
    cleaned_text = re.sub('\n+', '\n', cleaned_text)
    cleaned_text = re.sub('-+', '-', cleaned_text)
    cleaned_text = re.sub('```', '', cleaned_text)
    cleaned_text = re.sub('\n +', '\n', cleaned_text)
    cleaned_text = re.sub('\n+', '\n', cleaned_text)
    cleaned_text = re.sub('\*', '', cleaned_text)
    cleaned_text = re.sub('-', '', cleaned_text)
    cleaned_text = re.sub('\n', '', cleaned_text)
    cleaned_text = re.sub(r"(?<=[a-zA-Z]) (?=[A-Z])", ". ", cleaned_text)
    return cleaned_text

def remove_sentences_with_phrase(text, phrase):
    sentences = sent_tokenize(text)
    filtered_sentences = [sentence for sentence in sentences if phrase not in sentence]
    return ' '.join(filtered_sentences)

text_list = []
text_content = ""
phrase = "your doctor"
for _, row in combined_data.iterrows():
    row['AI Answers'] = clean_text(row['AI Answers'])
    row['AI Answers'] = remove_sentences_with_phrase(row['AI Answers'], phrase)
    row['Answer'] = clean_text(row['Answer'])
    text_list.append("Question: {}\nAnswer: {}\nThis is generated by 人\n\n".format(
        row['Question'], row['Answer']))
    text_list.append("Question: {}\nAnswer: {}\nThis is generated by 机\n\n".format(
        row['Question'], row['AI Answers']))

shuffled_text_list = sorted(text_list, key=lambda x: random.random())
for i in shuffled_text_list:
    text_content += i

text_file_path = '/content/training_data.txt'
with open(text_file_path, 'w', encoding='utf-8') as file:
    file.write(text_content)

text_file_path

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


'/content/training_data.txt'

In [ ]:
test_text_content = ""
for _, row in test_set.iterrows():
    row['AI Answers'] = clean_text(row['AI Answers'])
    row['Answer'] = clean_text(row['Answer'])
    test_text_content += "Question: {}\nAnswer: {}\n\n".format(
        row['Question'], row['Answer'])
    test_text_content += "Question: {}\nAnswer: {}\n\n".format(
        row['Question'], row['AI Answers'])

test_text_file_path = '/content/test_data.txt'
with open(test_text_file_path, 'w', encoding='utf-8') as file:
    file.write(test_text_content)

test_text_file_path

'/content/test_data.txt'